<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/pyLiBELa_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First things first
### From now on, we will be only using CUDA, a GPU computing API.
### In order to do that, we need to use the GPU runtime from Google Colab that can be selected following the steps in the menu:
### `Runtime > Change Runtime Type > T4 GPU`

In [1]:
#@title Downloading dependencies {display-mode: "form"}

%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
!pip install ipython-autotime
%load_ext autotime
!pip3 install condacolab
import condacolab
condacolab.install()
!mamba install openbabel


time: 47.5 s (started: 2023-11-17 17:17:55 +00:00)


In [1]:
#@title Installing pyLiBELa {display-mode: "form"}

%%capture
use_cuda = True #@param {type:"boolean"}

! rm -f Makefile*
! rm -rf obj src test
! rmdir obj src
if use_cuda:
  ! git clone --branch alex-works https://github.com/alessandronascimento/pyLiBELa.git
  ! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/alex-works/Colabs/Makefile
else:
  ! git clone --branch main https://github.com/alessandronascimento/pyLiBELa.git
  ! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile

! mv pyLiBELa/src src
! mv pyLiBELa/test test
! rm -rf pyLiBELa
! mkdir -p obj
! sed -i 's+-I/usr/include/openbabel3+-I/usr/local/include/openbabel3+g' Makefile
! make -j2


In [2]:
#@title Importing pyLiBELa {display-mode: "form"}
try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')

pyLiBELa is imported!


In [3]:
#@title Docking parameters {display-mode: "form"}

import os
import timeit
import numpy as np

Input = PARSER()

Input.generate_conformers = True;
Input.dock_parallel = False;
Input.parallel_jobs = 1;
Input.write_grids = False;
Input.load_grid_from_file = True;
Input.use_grids = True
Input.write_mol2 = True
Input.atom_limit = 10000 #@param {type:"number"}

scoring_function = "0" #@param ["0", "1", "2", "3"]
Input.dielectric_model = "r" #@param ["r", "constant"]
#Input.diel = 2.0;
Input.scoring_function = int(scoring_function)
grid_dimension = 30.0 #@param {type:"number"}
Input.grid_prefix = '../grids/McGrid'
Input.grid_spacing = 0.4 #@param {type:"number"}
Input.solvation_alpha = 0.1 #@param {type:"number"}
Input.solvation_beta = -0.005 #@param {type:"number"}

# Optimization parameter:
Input.min_tol = 1E-4;
Input.min_delta = 1E-4;
Input.dock_min_tol = 1E-4;
search_box = 8.0 #@param {type:"number"}
Input.timeout = 20 #@param {type:"number"}
Input.min_timeout = 30 #@param {type:"number"}
Input.overlay_optimizer = "mma" #@param ["mma", "ln_auglag", "subplex", "none"]
Input.energy_optimizer = "mma" #@param ["direct", "isres", "crs", "esch", "stogo", "mma", "simplex", "none"]
if (Input.scoring_function < 3):
    delta = 2.5 #@param {type:"number"}
    Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
    delta_es = 2.5 #@param {type:"number"}
    Input.deltaij_es6 = pow(delta_es, 6);
    Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.conf_search_trials = 10000;
Input.conformers_to_evaluate = 2;
Input.lig_conformers = 10;

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;

In [4]:
#@title Getting SB2021 data {display-mode: "form"}

%%capture
from google.colab import drive
drive.mount('/content/drive/')
sb_folder = '/content/drive/MyDrive/pyLiBELa/SB/' #@param {type:"string"}

with open(sb_folder+'list.txt') as f:
    targets_list = f.readlines()


targets_list = [target[0:4] for target in targets_list]



In [152]:
#@title Selecting the target from SB2021
%cd /content

#%cd /root/content

index_target=100 #@param {type:"integer"}
target=targets_list[index_target]
print("Selected target:", target)

folder = sb_folder+target

rec_file = folder + "/" +  target + ".rec.clean.mol2.gz"
lig_file = folder + "/" +  target + ".lig.am1bcc.mol2.gz"

!mkdir -p $target

rec_new = target+'/rec.mol2.gz'
lig_new = target+'/lig.mol2.gz'


%cp $rec_file $rec_new
%cp $lig_file $lig_new


/content
Selected target: 1D09
time: 821 ms (started: 2023-11-17 18:55:47 +00:00)


In [153]:

#@title Reading molecular files {display-mode: "form"}

#Reading Ligand SMILES


ligand_smiles = "N1C=Cc2ccccc12" #@param {type:"string"}


Lig2 = Mol2()
if Mol2.parse_smiles(Lig2, Input, ligand_smiles, 'Lig2'):
   print('Ligand SMILES parsed successfully!')
   print(ligand_smiles)

else:
  print('Oops, something wrong with you ligand... \nIt may be way too big or have an unknown atom.')

#Reading reference ligand and receptor from source

lig_src =  target + '/lig.mol2.gz'
rec_src = target + '/rec.mol2.gz'


REC = Mol2(Input, rec_src)
RefLig = Mol2(Input, lig_src)
#Lig2 = Mol2(Input,lig_src)

print('Receptor and reference ligand parsed successfully!')
print("Receptor has %4d atoms." % REC.N)
print('Reference Ligand has %4d atoms' % RefLig.N)
print('Search ligand has %4d atoms' % Lig2.N)


Ligand SMILES parsed successfully!
N1C=Cc2ccccc12
Receptor and reference ligand parsed successfully!
Receptor has 7261 atoms.
Reference Ligand has   22 atoms
Search ligand has   16 atoms
time: 79.8 ms (started: 2023-11-17 18:55:51 +00:00)


In [154]:
#@title Grid Generation and Docking calculation
%load_ext autotime
!rm -rf McLiBELa_dock.mol2.gz
import timeit
%cd $target

Writer = WRITER(Input)
Coord = COORD_MC()
HB = FindHB()
Ene  = Energy2(Input)

for i in range(len(REC.residue_pointer)-1):
  HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);
HB.find_ligandHB(lig_src, RefLig);
print('The receptor has %5d / %5d HB donors/acceptors around the active site.' % (len(REC.HBdonors), len(REC.HBacceptors)));

Dock = Docker(Writer)
center = Coord.compute_com(RefLig)

print()
start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);
print('Starting energy: %7.3f kcal/mol' % start_energy);
print()
print('Generating grids. This may take a while..')

start_time = timeit.default_timer()
Grids = Grid(Input, Writer, REC, center)
time = timeit.default_timer() - start_time

time_per_atom = time/REC.N
print('Grids computed, and it took %.2f s! That means %.4f s per atom.' %(time,time_per_atom))
grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);
print('Grid original energy: %7.3f kcal/mol' % grid_energy);
print('Grid error: %7.3f' % abs(100.*(start_energy-grid_energy)/start_energy));
print()
print()
print('Starting docking calculation...')
Dock.run(REC, Lig2, RefLig, center, Input, Grids, 0)
print('Docking calculation finished!')

Writer.write_box(center, Grids.xbegin, Grids.ybegin, Grids.zbegin, Grids.xend, Grids.yend, Grids.zend)


The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
/content/1D09
The receptor has    31 /    19 HB donors/acceptors around the active site.

Starting energy: -100.121 kcal/mol

Generating grids. This may take a while..
Grids computed, and it took 9.46 s! That means 0.0013 s per atom.
Grid original energy: -98.743 kcal/mol
Grid error:   1.377


Starting docking calculation...
Docking calculation finished!
time: 9.7 s (started: 2023-11-17 18:55:54 +00:00)


In [155]:
#@title Preparing Output Files
!rm -rf lig2.pdb lig_ref.pdb rec.pdb

#trocar aqui
!obabel -imol2 McLiBELa_dock.mol2.gz -opdb -O lig2.pdb
!obabel -imol2 lig.mol2.gz -opdb -O lig_ref.pdb
!obabel -imol2 rec.mol2.gz -opdb -O rec.pdb

lig2_pdb = open('lig2.pdb', 'r').read()
lig_ref_pdb = open('lig_ref.pdb', 'r').read()
rec_pdb = open('rec.pdb','r').read()

box_pdb = open('box.pdb','r').read()

def join_pdb(name_file_rec,name_file_lig,name_file_merge):
  !cp $name_file_rec temp_rec.pdb
  !cp $name_file_lig temp_lig.pdb

  !sed -i '/END\|CONECT/d' temp_rec.pdb #tira o end e o conect do arquivo do receptor

  !sed -i 's/A  /B  /g' temp_lig.pdb #troca a cadeia A por B no arquivo do ligante

  !sed -i '/CONECT/d' temp_lig.pdb #tira o conect do arquivo do ligante

  !cat temp_rec.pdb temp_lig.pdb > $name_file_merge #concatena os arquivos

  !rm -rf temp_rec.pdb temp_lig.pdb


join_pdb('rec.pdb','lig_ref.pdb','complex1.pdb')
complex1_pdb = open('complex1.pdb','r').read()

join_pdb('rec.pdb','lig2.pdb','complex2.pdb')
complex2_pdb = open('complex2.pdb','r').read()


1 molecule converted
1 molecule converted
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is SMALL)

1 molecule converted
time: 1.98 s (started: 2023-11-17 18:56:34 +00:00)


In [156]:
#@title Setting view
%%capture
try:
  import py3Dmol
except:
  !pip install py3Dmol
  import py3Dmol
aminoacids = ['MET', 'THR', 'ASN', 'LYS', 'SER', 'ARG', 'VAL', 'ALA', 'ASP', 'GLU', 'GLY', 'PHE', 'LEU', 'TYR', 'CYS', 'TRP', 'PRO', 'HIS', 'GLN', 'ILE', 'ASH', 'CYX', 'GLH', 'HIE', 'HID', 'HIP','HOH']


time: 709 µs (started: 2023-11-17 18:56:42 +00:00)


In [163]:
#@title Docked complex
#@markdown Do you want to show the grid box?
show_box = True #@param {type:"boolean"}
#@markdown Do you want to show the reference ligand? If selected, it will be shown in green.
show_ref = True #@param {type:"boolean"}
#@markdown Do you want to show the target protein?
show_rec = True #@param {type:"boolean"}
#@markdown Do you want to show the search ligand? If selected, it will be shown in yellow.
show_lig2 = True #@param {type:"boolean"}
#@markdown Do you want to show residues from the protein close to the search ligand?
show_close_res = True #@param {type:"boolean"}
#@markdown How close do you want them to be? Put the radius in Angstroms
dist=5 #@param {type:"integer"}


view = py3Dmol.view(width=800,height=600)

model=0

if show_box:
  view.addModel(box_pdb,'pdb')
  view.setStyle({'model':model}, {'stick':{"color":"#ff0000","width":2}})
  print(model)
  model+=1

if show_ref:
  view.addModel(lig_ref_pdb,'pdb')
  view.setStyle({'model':model},{'stick': {'colorscheme':'greenCarbon','opacity':0.7}})
  model+=1

if show_rec:

  view.addModel(complex2_pdb,'pdb')
  view.setStyle({'model':model,'chain':'B'},{'opacity':0})
  view.setStyle({'model':model,'chain':'A'},  {"cartoon":{'arrows':True,'colorscheme':'ssPymol','opacity':0.7}})
  view.setStyle({ 'model':model,'chain':'A', 'not':{'resn':aminoacids}}, {'sphere':{}})

  if show_close_res:
    view.addStyle({'model':model,'chain':'A','within':{'distance':'%.d'%dist, 'sel':{'chain':'B'}},'byres':True}, {'stick': {'colorscheme':'skyblueCarbon'}})

  if show_lig2:
      view.setStyle({'model':model,'chain':'B'},{'stick': {'colorscheme':'yellowCarbon'}})
  model+=1

else:
  if show_lig2:
    view.addModel(lig2_pdb,'pdb')
    view.setStyle({'model':model},{'stick': {'colorscheme':'yellowCarbon'}})
    model+=1
  if show_close_res:
    print('To show the residues close to the ligand, you need to select the show_rec box.')


view.setBackgroundColor('white')

view.zoomTo()
view.show()

0


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

time: 34.5 ms (started: 2023-11-17 18:58:39 +00:00)
